## This files combines the basic structures and logic of the implementation of user study, sequence is not correct, the final version will be in another file 

In [18]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [19]:
from ast import literal_eval
from os import listdir
from os.path import isfile, join
from scipy.sparse import csr_matrix, load_npz, save_npz
from tqdm import tqdm
import statistics as stats
import seaborn as sns
import datetime
import json
import numpy as np
import pandas as pd
import time
import yaml
import scipy.sparse as sparse
import yaml
from tkinter import *
import tkinter
from geopy.geocoders import Nominatim
from geopy import distance
from geopy import Point

## Dataset files 

In [20]:
#reviewJson = "..\\data\\Export_CleanedReview.json"
#reviewJsonWithClosedRes = "..\\data\\Export_CleanedReviewWithClosedRes.json"
reviewJsonToronto = "..\\data\\Export_TorontoData.json"


## Load Data
Select top frenquent user and top frequenty restaurants that had at least 1 review >= 4 stars (Kickking out users that gave all  reviews <=3 and restaurants that never got start >= 4 stars)

In [21]:
def get_yelp_df(path = 'data/', filename = 'Export_CleanedReview.json', sampling=False, top_user_num=6100, top_item_num=4000):
    """
    Get the pandas dataframe
    Sampling only the top users/items by density 
    Implicit representation applies
    """
    with open(filename,'r') as f:
        data = f.readlines()
        data = list(map(json.loads, data))
    
    data = data[0]
    #Get all the data from the dggeata file
    df = pd.DataFrame(data)
    
    df.rename(columns={'stars': 'review_stars', 'text': 'review_text', 'cool': 'review_cool',
                       'funny': 'review_funny', 'useful': 'review_useful'},
              inplace=True)

    df['business_num_id'] = df.business_id.astype('category').\
        cat.rename_categories(range(0, df.business_id.nunique()))
    df['business_num_id'] = df['business_num_id'].astype('int')

    df['user_num_id'] = df.user_id.astype('category').\
    cat.rename_categories(range(0, df.user_id.nunique()))
    df['user_num_id'] = df['user_num_id'].astype('int')

    df['timestamp'] = df['date'].apply(date_to_timestamp)

    if sampling:
        df = filter_yelp_df(df, top_user_num=top_user_num, top_item_num=top_item_num)
        # Refresh num id
        df['business_num_id'] = df.business_id.astype('category').\
        cat.rename_categories(range(0, df.business_id.nunique()))
        df['business_num_id'] = df['business_num_id'].astype('int')
        
        df['user_num_id'] = df.user_id.astype('category').\
        cat.rename_categories(range(0, df.user_id.nunique()))
        df['user_num_id'] = df['user_num_id'].astype('int')
#     drop_list = ['date','review_id','review_funny','review_cool','review_useful']
#     df = df.drop(drop_list, axis=1)

    df = df.reset_index(drop = True)

    return df 

def filter_yelp_df(df, top_user_num=6100, top_item_num=4000):
    #Getting the reviews where starts are above 3
    df_implicit = df[df['review_stars']>3]
    frequent_user_id = df_implicit['user_num_id'].value_counts().head(top_user_num).index.values
    frequent_item_id = df_implicit['business_num_id'].value_counts().head(top_item_num).index.values
    return df.loc[(df['user_num_id'].isin(frequent_user_id)) & (df['business_num_id'].isin(frequent_item_id))]

def date_to_timestamp(date):
    dt = datetime.datetime.strptime(date, '%Y-%m-%d')
    return time.mktime(dt.timetuple())

def df_to_sparse(df, row_name='userId', col_name='movieId', value_name='rating',
                 shape=None):
    rows = df[row_name]
    cols = df[col_name]
    if value_name is not None:
        values = df[value_name]
    else:
        values = [1]*len(rows)

    return csr_matrix((values, (rows, cols)), shape=shape)

## Get rating-UI and timestamp-UI matrix from original df

In [22]:
def get_rating_timestamp_matrix(df, sampling=False, top_user_num=6100, top_item_num=4000):
    """
    """
    #make the df implicit with top frenquent users and 
    #no need to sample anymore if df was sampled before 
    if sampling:
        df = filter_yelp_df(df, top_user_num=top_user_num, top_item_num=top_item_num)

    rating_matrix = df_to_sparse(df, row_name='user_num_id',
                                 col_name='business_num_id',
                                 value_name='review_stars',
                                 shape=None)
    
    #Have same dimension and data entries with rating_matrix, except that the review stars are - user avg
#     ratingWuserAvg_matrix = df_to_sparse(df, row_name='user_num_id',
#                                  col_name='business_num_id',
#                                  value_name='reviewStars_userAvg',
#                                  shape=None)
    
    timestamp_matrix = df_to_sparse(df, row_name='user_num_id',
                                    col_name='business_num_id',
                                    value_name='timestamp',
                                    shape=None)
    
    
    IC_matrix = get_I_C(df)
#     ratingWuserAvg_matrix
    return rating_matrix, timestamp_matrix, IC_matrix

In [23]:
def get_I_C(df):
    lst = df.categories.values.tolist()
    cat = []
    for i in range(len(lst)):
        cat.extend(lst[i].split(', '))
    unique_cat = set(cat)
    #     set categories id
    df_cat = pd.DataFrame(list(unique_cat),columns=["Categories"])
    df_cat['cat_id'] = df_cat.Categories.astype('category').cat.rename_categories(range(0, df_cat.Categories.nunique()))
    dict_cat = df_cat.set_index('Categories')['cat_id'].to_dict()
    
    df_I_C = pd.DataFrame(columns=['business_num_id', 'cat_id'])
    
    for i in range((df['business_num_id'].unique().shape)[0]):
        df_temp = df[df['business_num_id'] == i].iloc[:1]
        temp_lst = df_temp['categories'].to_list()[0].split(",")
        for j in range(len(temp_lst)):
            df_I_C = df_I_C.append({'business_num_id' : i  , 'cat_id' : dict_cat[temp_lst[j].strip()]} , ignore_index=True)
    
    IC_Matrix = df_to_sparse(df_I_C, row_name='business_num_id',
                                 col_name='cat_id',
                                 value_name=None,
                                 shape=None)    
    return IC_Matrix

## Time ordered split 

In [24]:
def time_ordered_split(rating_matrix, ratingWuserAvg_matrix, timestamp_matrix, ratio=[0.5, 0.2, 0.3],
                       implicit=True, remove_empty=False, threshold=3,
                       sampling=False, sampling_ratio=0.1):
    """
    Split the data to train,valid,test by time
    ratio:  train:valid:test
    threshold: for implicit representation
    """
    if implicit:
        temp_rating_matrix = sparse.csr_matrix(rating_matrix.shape)
        temp_rating_matrix[(rating_matrix > threshold).nonzero()] = 1
        rating_matrix = temp_rating_matrix
        timestamp_matrix = timestamp_matrix.multiply(rating_matrix)
        #ratingWuserAvg_matrix = ratingWuserAvg_matrix.multiply(rating_matrix)

    nonzero_index = None

    #Default false, not removing empty columns and rows
    #Should not have this case, since users should have at least 1 record of 4,5 
    #And restuarant should have at least 1 record of 4,5 
    if remove_empty:
        # Remove empty columns. record original item index
        nonzero_index = np.unique(rating_matrix.nonzero()[1])
        rating_matrix = rating_matrix[:, nonzero_index]
        timestamp_matrix = timestamp_matrix[:, nonzero_index]
        ratingWuserAvg_matrix = ratingWuserAvg_matrix[:, nonzero_index]

        # Remove empty rows. record original user index
        nonzero_rows = np.unique(rating_matrix.nonzero()[0])
        rating_matrix = rating_matrix[nonzero_rows]
        timestamp_matrix = timestamp_matrix[nonzero_rows]
        ratingWuserAvg_matrix = ratingWuserAvg_matrix[nonzero_rows]

    user_num, item_num = rating_matrix.shape

    rtrain = []
    rtrain_userAvg = []
    rtime = []
    rvalid = []
    rvalid_userAvg = []
    rtest = []
    rtest_userAvg = []
    # Get the index list corresponding to item for train,valid,test
    item_idx_train = []
    item_idx_valid = []
    item_idx_test = []
    
    for i in tqdm(range(user_num)):
        #Get the non_zero indexs, restuarants where the user visited/liked if implicit 
        item_indexes = rating_matrix[i].nonzero()[1]
        
        #Get the data for the user
        data = rating_matrix[i].data
        
        #Get time stamp value 
        timestamp = timestamp_matrix[i].data
        
        #Get review stars with user avg data 
        if implicit == False:
            dataWuserAvg = ratingWuserAvg_matrix[i].data
        
        #Non zero reviews for this user
        num_nonzeros = len(item_indexes)
        
        #If the user has at least one review
        if num_nonzeros >= 1:
            #Get number of test and valid data 
            #train is 30%
            num_test = int(num_nonzeros * ratio[2])
            #validate is 50%
            num_valid = int(num_nonzeros * (ratio[1] + ratio[2]))

            valid_offset = num_nonzeros - num_valid
            test_offset = num_nonzeros - num_test

            #Sort the timestamp for each review for the user
            argsort = np.argsort(timestamp)
            
            #Sort the reviews for the user according to the time stamp 
            data = data[argsort]
            
            #Sort the review with user avg accoridng to the time stamp
            if implicit == False:
                dataWuserAvg = dataWuserAvg[argsort]
            
            #Non-zero review index sort according to time
            item_indexes = item_indexes[argsort]
            
            #list of ratings, num of valid_offset index, index where there's non-zeros
            rtrain.append([data[:valid_offset], np.full(valid_offset, i), item_indexes[:valid_offset]])
            
            if implicit == False:
                #Changing valid set to binary
                count=valid_offset
                for eachData in data[valid_offset:test_offset]:
                    #if rating-avgRating > 0 then like
                    if eachData >= 4:
                        data[count] = 1
                    else:
                        data[count] = 0
                    count += 1
                
            #50%-70%
            rvalid.append([data[valid_offset:test_offset], np.full(test_offset - valid_offset, i),
                           item_indexes[valid_offset:test_offset]])
            #remaining 30%
            rtest.append([data[test_offset:], np.full(num_test, i), item_indexes[test_offset:]])
            
            if implicit == False:
                #Now for the rating matrix that considers user average rating
                #list of ratings, num of valid_offset index, index where there's non-zeros
                rtrain_userAvg.append([dataWuserAvg[:valid_offset], np.full(valid_offset, i), item_indexes[:valid_offset]])
                #50%-70%

                #Changing valid set to binary
                count=valid_offset
                for eachData in dataWuserAvg[valid_offset:test_offset]:
                    #if rating-avgRating > 0 then like
                    if eachData > 0:
                        dataWuserAvg[count] = 1
                    else:
                        dataWuserAvg[count] = 0
                    count += 1

                rvalid_userAvg.append([dataWuserAvg[valid_offset:test_offset], np.full(test_offset - valid_offset, i),
                               item_indexes[valid_offset:test_offset]])

                #Change test set to binary even we don't use it
                countTest = test_offset
                for eachData in dataWuserAvg[test_offset:]:
                    #if rating-avgRating > 0 then like
                    if eachData > 0:
                        dataWuserAvg[count] = 1
                    else:
                        dataWuserAvg[count] = 0
                    count += 1


                #remaining 30%
                rtest_userAvg.append([dataWuserAvg[test_offset:], np.full(num_test, i), item_indexes[test_offset:]])
                
            item_idx_train.append(item_indexes[:valid_offset])
            
#             item_idx_valid.append(item_indexes[valid_offset:test_offset])
#             item_idx_test.append(item_indexes[test_offset:])
        else:
            item_idx_train.append([])
#             item_idx_valid.append([])
#             item_idx_test.append([])
    
    rtrain = np.array(rtrain)
    rvalid = np.array(rvalid)
    rtest = np.array(rtest)
    if implicit == False:
        rtrain_userAvg = np.array(rtrain_userAvg)
        rvalid_userAvg = np.array(rvalid_userAvg)
        rtest_userAvg = np.array(rtest_userAvg)

    #take non-zeros values, row index, and column (non-zero) index and store into sparse matrix
    rtrain = sparse.csr_matrix((np.hstack(rtrain[:, 0]), (np.hstack(rtrain[:, 1]), np.hstack(rtrain[:, 2]))),
                               shape=rating_matrix.shape, dtype=np.float32)
    rvalid = sparse.csr_matrix((np.hstack(rvalid[:, 0]), (np.hstack(rvalid[:, 1]), np.hstack(rvalid[:, 2]))),
                               shape=rating_matrix.shape, dtype=np.float32)
    rtest = sparse.csr_matrix((np.hstack(rtest[:, 0]), (np.hstack(rtest[:, 1]), np.hstack(rtest[:, 2]))),
                              shape=rating_matrix.shape, dtype=np.float32)
    
    if implicit == False:
        rtrain_userAvg = sparse.csr_matrix((np.hstack(rtrain_userAvg[:, 0]), (np.hstack(rtrain_userAvg[:, 1]), np.hstack(rtrain_userAvg[:, 2]))),
                                   shape=rating_matrix.shape, dtype=np.float32)
        rvalid_userAvg = sparse.csr_matrix((np.hstack(rvalid_userAvg[:, 0]), (np.hstack(rvalid_userAvg[:, 1]), np.hstack(rvalid_userAvg[:, 2]))),
                                   shape=rating_matrix.shape, dtype=np.float32)
        rtest_userAvg = sparse.csr_matrix((np.hstack(rtest_userAvg[:, 0]), (np.hstack(rtest_userAvg[:, 1]), np.hstack(rtest_userAvg[:, 2]))),
                                  shape=rating_matrix.shape, dtype=np.float32)

    return rtrain, rvalid, rtest,rtrain_userAvg, rvalid_userAvg, rtest_userAvg, nonzero_index, timestamp_matrix, item_idx_train, item_idx_valid, item_idx_test

In [25]:
def time_ordered_splitModified(rating_matrix, ratingWuserAvg_matrix, timestamp_matrix, ratio=[0.5, 0.2, 0.3],
                       implicit=True, remove_empty=False, threshold=3,
                       sampling=False, sampling_ratio=0.1, trainSampling=1):
    """
    Split the data to train,valid,test by time
    ratio:  train:valid:test
    threshold: for implicit representation
    """
    
    
    if implicit:
        temp_rating_matrix = sparse.csr_matrix(rating_matrix.shape)
        temp_rating_matrix[(rating_matrix > threshold).nonzero()] = 1
        rating_matrix = temp_rating_matrix
        timestamp_matrix = timestamp_matrix.multiply(rating_matrix)
        #ratingWuserAvg_matrix = ratingWuserAvg_matrix.multiply(rating_matrix)

    nonzero_index = None

    #Default false, not removing empty columns and rows
    #Should not have this case, since users should have at least 1 record of 4,5 
    #And restuarant should have at least 1 record of 4,5 
    if remove_empty:
        # Remove empty columns. record original item index
        nonzero_index = np.unique(rating_matrix.nonzero()[1])
        rating_matrix = rating_matrix[:, nonzero_index]
        timestamp_matrix = timestamp_matrix[:, nonzero_index]
        ratingWuserAvg_matrix = ratingWuserAvg_matrix[:, nonzero_index]

        # Remove empty rows. record original user index
        nonzero_rows = np.unique(rating_matrix.nonzero()[0])
        rating_matrix = rating_matrix[nonzero_rows]
        timestamp_matrix = timestamp_matrix[nonzero_rows]
        ratingWuserAvg_matrix = ratingWuserAvg_matrix[nonzero_rows]

    user_num, item_num = rating_matrix.shape

    rtrain = []
    rtrain_userAvg = []
    rtime = []
    rvalid = []
    rvalid_userAvg = []
    rtest = []
    rtest_userAvg = []
    # Get the index list corresponding to item for train,valid,test
    item_idx_train = []
    item_idx_valid = []
    item_idx_test = []
    
    for i in tqdm(range(user_num)):
        #Get the non_zero indexs, restuarants where the user visited/liked if implicit 
        item_indexes = rating_matrix[i].nonzero()[1]        
        #Get the data for the user
        data = rating_matrix[i].data      
        #Get time stamp value 
        timestamp = timestamp_matrix[i].data 
        #Get review stars with user avg data 
        if implicit == False:
            dataWuserAvg = ratingWuserAvg_matrix[i].data

            
        #Non zero reviews for this user
        num_nonzeros = len(item_indexes)
        
        #If the user has at least one review
        if num_nonzeros >= 1:
            num_test = int(num_nonzeros * ratio[2])
            num_valid = int(num_nonzeros * (ratio[1] + ratio[2]))
            valid_offset = num_nonzeros - num_valid
            
            # Adding this for sampling for training set
            valid_offsetSample = int(valid_offset*trainSampling)
            test_offset = num_nonzeros - num_test
            
            #Sort the timestamp for each review for the user
            argsort = np.argsort(timestamp)
            
            #Sort the reviews for the user according to the time stamp 
            data = data[argsort]
            
            #Sort the review with user avg accoridng to the time stamp
            if implicit == False:
                dataWuserAvg = dataWuserAvg[argsort]
            
            #Non-zero review index sort according to time
            item_indexes = item_indexes[argsort]
            
            #list of ratings, num of valid_offset index, index where there's non-zeros
            #if take from old to new
            #rtrain.append([data[:valid_offsetSample], np.full(valid_offsetSample, i), item_indexes[:valid_offsetSample]])
            #if take from new to old
            rtrain.append([data[valid_offset-valid_offsetSample:valid_offset], np.full(valid_offsetSample, i), item_indexes[valid_offset-valid_offsetSample:valid_offset]])
            rvalid.append([data[valid_offset:test_offset], np.full(test_offset - valid_offset, i),
                           item_indexes[valid_offset:test_offset]])
            rtest.append([data[test_offset:], np.full(num_test, i), item_indexes[test_offset:]])
            
            if implicit == False:
                #Now for the rating matrix that considers user average rating
                #list of ratings, num of valid_offset index, index where there's non-zeros
                #from old to new
                #rtrain_userAvg.append([dataWuserAvg[:valid_offsetSample], np.full(valid_offsetSample, i), item_indexes[:valid_offsetSample]])
                #take nearest
                rtrain_userAvg.append([dataWuserAvg[valid_offset-valid_offsetSample:valid_offset], np.full(valid_offsetSample, i), item_indexes[valid_offset-valid_offsetSample:valid_offset]])                
                    
                rvalid_userAvg.append([dataWuserAvg[valid_offset:test_offset], np.full(test_offset - valid_offset, i),
                               item_indexes[valid_offset:test_offset]])
                
                rtest_userAvg.append([dataWuserAvg[test_offset:], np.full(num_test, i), item_indexes[test_offset:]])
                
            item_idx_train.append(item_indexes[:valid_offsetSample])
            
        else:
            item_idx_train.append([])
    
    rtrain = np.array(rtrain)
    rvalid = np.array(rvalid)
    rtest = np.array(rtest)
   
    if implicit == False:
        rtrain_userAvg = np.array(rtrain_userAvg)
        rvalid_userAvg = np.array(rvalid_userAvg)
        rtest_userAvg = np.array(rtest_userAvg)

    #take non-zeros values, row index, and column (non-zero) index and store into sparse matrix
    rtrain = sparse.csr_matrix((np.hstack(rtrain[:, 0]), (np.hstack(rtrain[:, 1]), np.hstack(rtrain[:, 2]))),
                               shape=rating_matrix.shape, dtype=np.float32)
    rvalid = sparse.csr_matrix((np.hstack(rvalid[:, 0]), (np.hstack(rvalid[:, 1]), np.hstack(rvalid[:, 2]))),
                               shape=rating_matrix.shape, dtype=np.float32)
    rtest = sparse.csr_matrix((np.hstack(rtest[:, 0]), (np.hstack(rtest[:, 1]), np.hstack(rtest[:, 2]))),
                              shape=rating_matrix.shape, dtype=np.float32)
    
    if implicit == False:
        rtrain_userAvg = sparse.csr_matrix((np.hstack(rtrain_userAvg[:, 0]), (np.hstack(rtrain_userAvg[:, 1]), np.hstack(rtrain_userAvg[:, 2]))),
                                   shape=rating_matrix.shape, dtype=np.float32)
        rvalid_userAvg = sparse.csr_matrix((np.hstack(rvalid_userAvg[:, 0]), (np.hstack(rvalid_userAvg[:, 1]), np.hstack(rvalid_userAvg[:, 2]))),
                                   shape=rating_matrix.shape, dtype=np.float32)
        rtest_userAvg = sparse.csr_matrix((np.hstack(rtest_userAvg[:, 0]), (np.hstack(rtest_userAvg[:, 1]), np.hstack(rtest_userAvg[:, 2]))),
                                  shape=rating_matrix.shape, dtype=np.float32)

    return rtrain, rvalid, rtest,rtrain_userAvg, rvalid_userAvg, rtest_userAvg, nonzero_index, timestamp_matrix, item_idx_train, item_idx_valid, item_idx_test


## Get df for training corpus

In [26]:
#Item idex matrix stores the reivews starts
#This function returns a list of index for the reviews included in training set 
def get_corpus_idx_list(df, item_idx_matrix):
    """
    Input: 
    df: total dataframe
    item_idx_matrix: train index list got from time_split 
    Output: row index in original dataframe for training data by time split
    """
    lst = []
    #For all the users: 5791
    for i in tqdm(range(len(item_idx_matrix))):
        
        #find row index where user_num_id is i
        a = df.index[df['user_num_id'] == i].tolist()
        
        #loop through the busienss id that the user i reviewed for in offvalid set 
        for item_idx in  item_idx_matrix[i]:
            
            #get the row index for reviews for business that the user liked in the train set
            b = df.index[df['business_num_id'] == item_idx].tolist()
            
            #Find the index for which this user liked, one user only rate a business once
            idx_to_add = list(set(a).intersection(b))
            
            if idx_to_add not in lst:
                lst.extend(idx_to_add)
    return lst

## Preprocess using Term Frequency - CounterVectorizer

In [27]:
#Stemming and Lemmatisation
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
import re 
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import RegexpTokenizer
# Get corpus and CountVector
from sklearn.feature_extraction.text import CountVectorizer
nltk.download('wordnet')
nltk.download('stopwords')
lem = WordNetLemmatizer()
stem = PorterStemmer()
stop_words = set(stopwords.words("english"))
new_words = ['not_the']
stop_words = stop_words.union(new_words)

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\shenti10\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\shenti10\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [28]:
from tqdm import tqdm

#Should 'because' added?
def preprocess(df, reset_list = [',','.','?',';','however','but']):
    corpus = []
    for i in tqdm(range(df.shape[0])):
        text = df['review_text'][i]
        change_flg = 0
        #Convert to lowercase
        text = text.lower()
        
        ##Convert to list from string, loop through the review text
        text = text.split()
        
        #any sentence that encounters a not, the folloing words will become not phrase until hit the sentence end
        for j in range(len(text)):
            #Make the not_ hack
            if text[j] == 'not':
                change_flg = 1
#                 print 'changes is made after ', i
                continue
            #if was 1 was round and not hit a 'not' in this round
            if change_flg == 1 and any(reset in text[j] for reset in reset_list):
                text[j] = 'not_' + text[j]
                change_flg = 0
#                 print 'reset at ', i
            if change_flg == 1:
                text[j] = 'not_' + text[j]
        
        #Convert back to string
        text = " ".join(text)
        
        #Remove punctuations
#       text = re.sub('[^a-zA-Z]', ' ', text)
        
        #remove tags
        text=re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",text)
        
        # remove special characters and digits
        text=re.sub("(\\d|\\W)+"," ",text)
        
        ##Convert to list from string
        text = text.split()
        
        ##Stemming
        ps=PorterStemmer()
        
        #Lemmatisation
        lem = WordNetLemmatizer()
        text = [lem.lemmatize(word) for word in text if not word in  
                stop_words] 
        text = " ".join(text)
        corpus.append(text)
    return corpus

## All Model

In [29]:
from sklearn.metrics.pairwise import cosine_similarity
def train(matrix_train):
    similarity = cosine_similarity(X=matrix_train, Y=None, dense_output=True)
    return similarity

def get_I_K(df, X, row_name = 'business_num_id', binary = True, shape = (121994,6000)):
    """
    get the item-keyphrase matrix
    """
    rows = []
    cols = []
    vals = []
    #For each review history
    for i in tqdm(range(X.shape[0])):
        #Get the array of frequencies for document/review i 
        arr = X[i].toarray() 
        nonzero_element = arr.nonzero()[1]  # Get nonzero element in each line, keyphrase that appears index 
        length_of_nonzero = len(nonzero_element) #number of important keyphrase that appears
        
        # df[row_name][i] is the item idex
        #Get a list row index that indicates the document/review
        rows.extend(np.array([df[row_name][i]]*length_of_nonzero)) ## Item index
        #print(rows)
        
        #Get a list of column index indicating the key phrase that appears in i document/review
        cols.extend(nonzero_element) ## Keyword Index
        if binary:
            #Create a bunch of 1s
            vals.extend(np.array([1]*length_of_nonzero))
        else:
            #If not binary 
            vals.extend(arr[arr.nonzero()])    
    return csr_matrix((vals, (rows, cols)), shape=shape)


#Get a UI matrix if it's not item_similarity based or else IU
def predict(matrix_train, k, similarity, item_similarity_en = False):
    prediction_scores = []
    
    #inverse to IU matrix
    if item_similarity_en:
        matrix_train = matrix_train.transpose()
        
    #for each user or item, depends UI or IU 
    for user_index in tqdm(range(matrix_train.shape[0])):
        # Get user u's prediction scores for all items
        #Get prediction/similarity score for each user 1*num or user or num of items
        vector_u = similarity[user_index]

        # Get closest K neighbors excluding user u self
        #Decending accoding to similarity score, select top k
        similar_users = vector_u.argsort()[::-1][1:k+1]
        
        # Get neighbors similarity weights and ratings
        similar_users_weights = similarity[user_index][similar_users]
        
        #similar_users_weights_sum = np.sum(similar_users_weights)
        #print(similar_users_weights.shape)
        #shape: num of res * k
        similar_users_ratings = matrix_train[similar_users].toarray()
              
        prediction_scores_u = similar_users_ratings * similar_users_weights[:, np.newaxis]
        #print(prediction_scores_u)
        
        
        prediction_scores.append(np.sum(prediction_scores_u, axis=0))
        
    res = np.array(prediction_scores)
    
    if item_similarity_en:
        res = res.transpose()
    return res


#Preidction score is UI or IU?
def prediction(prediction_score, topK, matrix_Train):

    prediction = []

    #for each user
    for user_index in tqdm(range(matrix_Train.shape[0])):
        
        #take the prediction scores for user 1 * num res
        vector_u = prediction_score[user_index]
        
        #The restuarant the user rated
        vector_train = matrix_Train[user_index]
        
        if len(vector_train.nonzero()[0]) > 0:
            vector_predict = sub_routine(vector_u, vector_train, topK=topK)
        else:
            vector_predict = np.zeros(topK, dtype=np.float32)

        prediction.append(vector_predict)

    return np.vstack(prediction)

#topK: the number of restuarants we are suggesting 
#if vector_train has number, then the user has visited
def sub_routine(vector_u, vector_train, topK=500):

    #index where non-zero
    train_index = vector_train.nonzero()[1]
    
    vector_u = vector_u
    
    #get topk + num rated res prediction score descending, top index 
    candidate_index = np.argpartition(-vector_u, topK+len(train_index))[:topK+len(train_index)]
    
    #sort top prediction score index in range topK+len(train_index) into vector_u`
    vector_u = candidate_index[vector_u[candidate_index].argsort()[::-1]]
    
    #deleted the rated res from the topk+train_index prediction score vector for user u 
    #Delete the user rated res index from the topk+numRated index
    vector_u = np.delete(vector_u, np.isin(vector_u, train_index).nonzero()[0])

    #so we only include the top K prediction score here
    return vector_u[:topK]

## Evaluation

In [30]:
def recallk(vector_true_dense, hits, **unused):
    hits = len(hits.nonzero()[0])
    return float(hits)/len(vector_true_dense)


def precisionk(vector_predict, hits, **unused):
    hits = len(hits.nonzero()[0])
    return float(hits)/len(vector_predict)


def average_precisionk(vector_predict, hits, **unused):
    precisions = np.cumsum(hits, dtype=np.float32)/range(1, len(vector_predict)+1)
    return np.mean(precisions)


def r_precision(vector_true_dense, vector_predict, **unused):
    vector_predict_short = vector_predict[:len(vector_true_dense)]
    hits = len(np.isin(vector_predict_short, vector_true_dense).nonzero()[0])
    return float(hits)/len(vector_true_dense)


def _dcg_support(size):
    arr = np.arange(1, size+1)+1
    return 1./np.log2(arr)


def ndcg(vector_true_dense, vector_predict, hits):
    idcg = np.sum(_dcg_support(len(vector_true_dense)))
    dcg_base = _dcg_support(len(vector_predict))
    dcg_base[np.logical_not(hits)] = 0
    dcg = np.sum(dcg_base)
    return dcg/idcg


def click(hits, **unused):
    first_hit = next((i for i, x in enumerate(hits) if x), None)
    if first_hit is None:
        return 5
    else:
        return first_hit/10


def evaluate(matrix_Predict, matrix_Test, metric_names =['R-Precision', 'NDCG', 'Precision', 'Recall', 'MAP'], atK = [5, 10, 15, 20, 50], analytical=False):
    """
    :param matrix_U: Latent representations of users, for LRecs it is RQ, for ALSs it is U
    :param matrix_V: Latent representations of items, for LRecs it is Q, for ALSs it is V
    :param matrix_Train: Rating matrix for training, features.
    :param matrix_Test: Rating matrix for evaluation, true labels.
    :param k: Top K retrieval
    :param metric_names: Evaluation metrics
    :return:
    """
    global_metrics = {
        #"R-Precision": r_precision,
        #"NDCG": ndcg,
        #"Clicks": click
    }

    local_metrics = {
        #"Precision": precisionk,
        #"Recall": recallk,
        "MAP": average_precisionk
    }

    output = dict()

    num_users = matrix_Predict.shape[0]

    for k in atK:

        local_metric_names = list(set(metric_names).intersection(local_metrics.keys()))
        results = {name: [] for name in local_metric_names}
        topK_Predict = matrix_Predict[:, :k]

        for user_index in tqdm(range(topK_Predict.shape[0])):
            vector_predict = topK_Predict[user_index]
            if len(vector_predict.nonzero()[0]) > 0:
                vector_true = matrix_Test[user_index]
                vector_true_dense = vector_true.nonzero()[1]
                hits = np.isin(vector_predict, vector_true_dense)

                if vector_true_dense.size > 0:
                    for name in local_metric_names:
                        results[name].append(local_metrics[name](vector_true_dense=vector_true_dense,
                                                                 vector_predict=vector_predict,
                                                                 hits=hits))
        results_summary = dict()
        if analytical:
            for name in local_metric_names:
                results_summary['{0}@{1}'.format(name, k)] = round(results[name],4)
        else:
            for name in local_metric_names:
                results_summary['{0}@{1}'.format(name, k)] = (round((np.average(results[name])),4),
                                                              round((1.96*np.std(results[name])/np.sqrt(num_users)),4))
        output.update(results_summary)

    global_metric_names = list(set(metric_names).intersection(global_metrics.keys()))
    results = {name: [] for name in global_metric_names}

    topK_Predict = matrix_Predict[:]

    for user_index in tqdm(range(topK_Predict.shape[0])):
        vector_predict = topK_Predict[user_index]

        if len(vector_predict.nonzero()[0]) > 0:
            vector_true = matrix_Test[user_index]
            vector_true_dense = vector_true.nonzero()[1]
            hits = np.isin(vector_predict, vector_true_dense)

            # if user_index == 1:
            #     import ipdb;
            #     ipdb.set_trace()

            if vector_true_dense.size > 0:
                for name in global_metric_names:
                    results[name].append(global_metrics[name](vector_true_dense=vector_true_dense,
                                                              vector_predict=vector_predict,
                                                              hits=hits))
    results_summary = dict()
    if analytical:
        for name in global_metric_names:
            results_summary[name] = round(results[name],4)
    else:
        for name in global_metric_names:
            results_summary[name] = (round(np.average(results[name]),4), round((1.96*np.std(results[name])/np.sqrt(num_users)),4))
    output.update(results_summary)

    return output

## Add 2 matrices

In [31]:
## Utility functions
#3906 restuarant, 3000 keyphrase, 5791 user 
def add_two_matrix(ratio, U_I_matrix,I_K_matrix, shape = (3906, 3000+5791)):
    # ratio determine Keywords/User in the matrix
    rows = []
    cols = []
    datas = []
    I_U_matrix = U_I_matrix.transpose()
    
    #for each restuarant
    for i in tqdm(range(I_K_matrix.shape[0])):
        #key phrase that this item has, column(key phrase) index
        nonzero1 = I_K_matrix[i].nonzero()
        
        #user that rated this item, column(user) index 
        nonzero2 = I_U_matrix[i].nonzero()
        
        #Trying to create a sparse matrix that stores 
        #index of restuarant for (K + U) times
        row = [i]*(len(nonzero1[1])+len(nonzero2[1]))
        
        #column index for key phrase and users that are non-zero
        col = nonzero1[1].tolist()+ nonzero2[1].tolist()
        
        
        data = [ratio]*len(nonzero1[1])+[1-ratio]*len(nonzero2[1]) # Binary representation of I-K/U matrix
        
        rows.extend(row)
        cols.extend(col)
        datas.extend(data)
    return csr_matrix( (datas,(rows,cols)), shape=shape )

def transfer_to_implicit(rating_matrix, threshold = 0):
    temp_rating_matrix = sparse.csr_matrix(rating_matrix.shape)
    temp_rating_matrix[(rating_matrix > threshold).nonzero()] = 1
    rating_matrix = temp_rating_matrix
    return rating_matrix

In [32]:
def getImplicitMatrix(sparseMatrix, threashold=0):
    temp_matrix = sparse.csr_matrix(sparseMatrix.shape)
    temp_matrix[(sparseMatrix > threashold).nonzero()] = 1
    return temp_matrix

In [33]:
def predictUU(matrix_train, k, chooseWeigthMethod, similarity1=None, similarity2=None, similarity3=None, similarity4=None, similarity5=None, item_similarity_en = False):
    prediction_scores = []
    #Convert from list to ndarray, add an axis
    if isinstance(chooseWeigthMethod, list):
        chooseWeigthMethod = np.array(chooseWeigthMethod)[:, np.newaxis]
   
    "make sure that when passing in chooseWeightMethod, the weight must be aligned with similarity metrices, even if set to None"
    "They should add to 1 as well"
    #inverse to IU matrix
    if item_similarity_en:
        matrix_train = matrix_train.transpose()
        
    #for each user or item, depends UI or IU 
    for user_index in tqdm(range(matrix_train.shape[0])):
    #for user_index in tqdm(range(10,20)):
        
        numberSimilarMatrix = 0
        # Get user u's prediction scores for all items 
        #Get prediction/similarity score for each user 1*num or user or num of items
        if similarity1 is not None:
            vector_u1 = similarity1[user_index]
            numberSimilarMatrix += 1
        else:
            vector_u1 = [0]*matrix_train.shape[0]
            
        if similarity2 is not None:
            vector_u2 = similarity2[user_index]
            numberSimilarMatrix += 1
        else:
            vector_u2 = [0]*len(vector_u1)
            
        if similarity3 is not None:
            vector_u3 = similarity3[user_index]
            numberSimilarMatrix += 1
        else:
            vector_u3 = [0]*len(vector_u1)
            
        if similarity4 is not None:
            vector_u4 = similarity4[user_index]
            numberSimilarMatrix += 1
        else:
            vector_u4 = [0]*len(vector_u1)
        
        if similarity5 is not None:
            vector_u5 = similarity5[user_index]
            numberSimilarMatrix += 1
        else:
            vector_u5 = [0]*len(vector_u1)
        
        #Temperary vector that stacks all 4 vectors together
        tempVector = np.array([vector_u1,vector_u2,vector_u3,vector_u4, vector_u5])
        
        if chooseWeigthMethod is None:
            #Get the similarity score from the first similarity matrix anyways 
            vector_u = vector_u1.copy()
            
        #If we are choosing the max, min, avg or similarity scores
        if chooseWeigthMethod is not None:
            if chooseWeigthMethod == 'max':
                vector_u = tempVector.max(axis=0)
            elif chooseWeigthMethod == 'min':
                vector_u = tempVector.min(axis=0)
            elif chooseWeigthMethod == 'average':
                vector_u = tempVector.mean(axis=0)
            elif isinstance(chooseWeigthMethod, np.ndarray):
                #Validate that number of weights passed in equals number of matrices
                #assert(len(chooseWeigthMethod) == numberSimilarMatrix)
                #Get the new combined similarity vector 
                weighted_u = tempVector * chooseWeigthMethod
                vector_u =np.sum(weighted_u, axis=0)
                #print((vector_u == vector_u4).sum())
                
        similar_users = vector_u.argsort()[::-1][1:k+1]
        
        # Get neighbors similarity weights and ratings
        #similar_users_weights = similarity1[user_index][similar_users]
        similar_users_weights = vector_u[similar_users]
        
        #similar_users_weights_sum = np.sum(similar_users_weights)
        #print(similar_users_weights.shape)
        #shape: num of res * k
        similar_users_ratings = matrix_train[similar_users].toarray()
              
        prediction_scores_u = similar_users_ratings * similar_users_weights[:, np.newaxis]
        #print(prediction_scores_u)
        
        prediction_scores.append(np.sum(prediction_scores_u, axis=0))
        
    res = np.array(prediction_scores)
    
    if item_similarity_en:
        res = res.transpose()
    return res

## Methods for UI 

In [34]:
#Enter the user number that we are trying to recommend for
#Enter the # of businesses that we want to recommend for them 
#Pass in the UI_Prediction matrix for users 
#userIndex parameter not used 
def constructResDictionary(userIndex, busIndexRange, UI_prediction):
    #Construct the dictionary for the recommended restaurants to display 
    dictionaryToConstruct = {}
    
    #Loop through the number of businesses 
    for busIndex in range(busIndexRange):
        #Get the business information for the recommended business 
        businessSeries = df[df["business_num_id"] == UI_prediction[busIndex]].iloc[0]
        #Get the business name 
        busName = businessSeries['name']
        
        #get the list of strings to generate the address information 
        address_generator = (str(w) for w in yaml.safe_load(businessSeries.location)['display_address'])
        busLocation = ', '.join(address_generator)
        bus_Price = businessSeries.price
        busStars = businessSeries.business_stars
        busReviewCount = businessSeries.review_count_y 
        category_generator = (str(s) for s in [cateDict['title'] for cateDict in yaml.safe_load(businessSeries.categories)])
        #busCategories = [cateDict['title'] for cateDict in yaml.safe_load(businessSeries.categories)]
        busCategories = ', '.join(category_generator)
        #Now add the restaurant to the dictionary
        dictionaryToConstruct[busName] = {'Address': busLocation,\
                                'Price': bus_Price,\
                                 'Star': busStars, \
                                 'Review Count': busReviewCount, \
                                 'Category': busCategories}
    return dictionaryToConstruct

#### Get original dataframe out of the review datastet

In [35]:
df = get_yelp_df(path ='', filename=reviewJsonToronto, sampling= True)

#### Get rating-UI matrix and timestepm-UI matrix

In [36]:
rating_matrix, timestamp_matrix , I_C_matrix = get_rating_timestamp_matrix(df)

# get ratingWuserAvg_matrix
rating_array = rating_matrix.toarray()
user_average_array = rating_array.sum(axis = 1)/np.count_nonzero(rating_array,axis = 1)
init_UI = np.zeros(rating_array.shape)
init_UI[rating_array.nonzero()] = 1
for i in range(user_average_array.shape[0]):
    init_UI[i] = init_UI[i] * (user_average_array[i]-0.001) 
user_average_array = init_UI
ratingWuserAvg_array = rating_array - user_average_array
ratingWuserAvg_matrix=sparse.csr_matrix(ratingWuserAvg_array)

#### Split to get rtrain-UI matrix and valid and test.. item_index_matrix_train

In [146]:
rtrain_implicit, rvalid_implicit, rtest_implicit, rtrain_userAvg_implicit, rvalid_userAvg_implicit, rtest_userAvg_implicit, nonzero_index, rtime, item_idx_matrix_train_implicit,item_idx_matrix_valid_implicit, item_idx_matrix_test_implicit = time_ordered_splitModified(rating_matrix=rating_matrix, ratingWuserAvg_matrix=ratingWuserAvg_matrix, timestamp_matrix=timestamp_matrix,
                                                                     ratio=[0.5,0.2,0.3],
                                                                     implicit=True,
                                                                     remove_empty=False, threshold=3,sampling=False, 
                                                                     sampling_ratio=0.1, trainSampling=0.95)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  del sys.path[0]
100%|████████████████████████████████████████████████████████████████████████████| 6100/6100 [00:01<00:00, 3101.55it/s]


In [147]:
rtrain, rvalid, rtest, rtrain_userAvg, rvalid_userAvg, rtest_userAvg, nonzero_index, rtime, item_idx_matrix_train,item_idx_matrix_valid, item_idx_matrix_test = time_ordered_splitModified(rating_matrix=rating_matrix, ratingWuserAvg_matrix=ratingWuserAvg_matrix, timestamp_matrix=timestamp_matrix,
                                                                     ratio=[0.5,0.2,0.3],
                                                                     implicit=False,
                                                                     remove_empty=False, threshold=3,
                                                                     sampling=False, sampling_ratio=0.1, 
                                                                     trainSampling=0.95)  

100%|████████████████████████████████████████████████████████████████████████████| 6100/6100 [00:02<00:00, 2623.05it/s]


# ATTENTION: I am combining rtrain+rvalid+rtest over here to use for user study

In [148]:
rtrain = rtrain + rvalid + rtest 
rtrain_implicit = rtrain_implicit + rvalid_implicit + rtest_implicit
rtrain_userAvg = rtrain_userAvg + rvalid_userAvg + rtest_userAvg

# I am modifing df_train to the whole df as well, since we are using the whole data set 

#### Get df shrink to df_train for rtrain

In [40]:
df_train = df.copy()

In [41]:
# #Get the list of row index for the training set 
# lst_train = get_corpus_idx_list(df, item_idx_matrix_train)

# # Get the training dataframe from the original dataframe
# df_train = df.loc[lst_train]

# #Resetting the index of the train data
# df_train = df_train.reset_index(drop=True)

# df_train.shape

In [42]:
pd.set_option('display.max_columns', None)
df_train['business_num_id'].max()

3997

#### If using term frequency only to compute corpus and X(review vs. terms) CounterVectorizer

In [43]:
# The entire corpus
#corpus = preprocess(df_train)
# X row: df_train row, column: key words frequency 
# When building the vocabulary ignore terms that have a document frequency strictly higher than the given threshold
#cv=CountVectorizer(max_df=0.9,stop_words=stop_words, max_features=5000, ngram_range=(1,1))
#X=cv.fit_transform(corpus)

##### If using TD-IDF to compute corpus and X (business vs. terms) TfIdfVectorizer

In [44]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [45]:
corpus = preprocess(df_train)

100%|████████████████████████████████████████████████████████████████████████| 187837/187837 [02:25<00:00, 1290.78it/s]


In [46]:
#Creating a dictionary to store business: review text
dict_text = {}
for i in range(len(corpus)):
    if df_train['business_num_id'][i] not in dict_text:
        dict_text[df_train['business_num_id'][i]] = corpus[i]
    else:
        temp = dict_text[df_train['business_num_id'][i]]
        temp = temp + corpus[i]
        dict_text[df_train['business_num_id'][i]] = temp

In [47]:
#Create a list for the review text, where the row dimension = total business ids
list_text = []
for key in range(0,max(list(dict_text.keys()))+1) :
    if key not in dict_text.keys():
        list_text.extend([""])
    else:
        list_text.extend([dict_text[key]])

In [48]:
#Get the X vector, where dimension is #business vs #terms
vectorizer = TfidfVectorizer(max_df=0.9,stop_words=stop_words, max_features=5000, ngram_range=(1,1))
X_cleaned = vectorizer.fit_transform(list_text).toarray()
X_cleaned_sparse = csr_matrix(X_cleaned)

## User-rating KNN

### 1. With ratings that subtracts user average rating, cosine similarity

In [49]:
#UU similarity, using cosine similarity
similarity_1 = train(rtrain)
#get a user-item matrix  UI prediction
#Predict using UI matrix with ratings in it 
UI_predictionScore_Explicit = predict(rtrain_userAvg, 100, similarity_1, item_similarity_en= False)
UI_predict_Explicit = prediction(UI_predictionScore_Explicit, 50, rtrain_userAvg)
#user_item_res1 = evaluate(user_item_predict1, rvalid_userAvg)

100%|████████████████████████████████████████████████████████████████████████████| 6100/6100 [00:02<00:00, 2989.07it/s]


In [50]:
UI_predict_Explicit[0]

array([3655, 2396, 3115, 1819, 1774, 3329, 3607,  593,  691,  440, 2669,
       1293, 2018, 2192, 3609, 3414,  302,  572, 1374,  667, 2294, 3309,
        989,  511, 1346,  101, 3166, 2900, 3073, 3542, 1771,  918, 2175,
       3909, 3042, 2715, 2856,  155, 3489,  192,  425,  966, 2277, 1582,
       2628,  631, 2156, 2771, 2892,  688], dtype=int64)

In [51]:
#Recommendation list if only use train data
# array([2396,  101, 2579, 2600, 1819, 3489,  355, 3607, 3542, 1374,  915,
#         845, 2018, 1912, 3982,  425, 3245, 1894,  375, 2906, 1083, 2295,
#         593, 2842, 1057,  629, 2525, 2538, 3629, 2277,  324, 2816, 3115,
#         107,  594, 3951,  302, 1927, 3631, 1935, 1131, 1884,  829, 3166,
#        2505, 1393, 3947,  636, 2856,  861], dtype=int64)

### 2. Implicit User-rating

In [52]:
#UU similarity, using cosine similarity
similarity_2 = train(rtrain_implicit)
UI_predictionScore_Implicit = predict(rtrain_implicit, 100, similarity_2, item_similarity_en= False)
UI_predict_Implicit = prediction(UI_predictionScore_Implicit, 50, rtrain_implicit)

100%|████████████████████████████████████████████████████████████████████████████| 6100/6100 [00:02<00:00, 2770.30it/s]


In [53]:
UI_predict_Implicit[0]

array([3655,  753, 2396, 2235, 1582,  101,  355, 2856,  883, 2733, 2663,
        691, 2600,  292, 2669, 3222, 1912, 2743, 1248, 2887, 2579, 2526,
       3564, 2460, 1914, 2361, 3245, 2156, 1555, 3672, 2728, 1910, 1725,
       2080, 3295, 1813, 2146, 2175, 2358, 3629, 3354, 1013,  779,  328,
       1800, 1829, 1212, 2547,  508, 3175], dtype=int64)

### 3. Implicit similarity, Explicit user-rating combination


In [54]:
similarity_3 = train(rtrain_implicit)
#get a user-item matrix  UI prediction
#Predict using UI matrix with ratings in it 
UI_predictionScore_IECombined = predict(rtrain, 100, similarity_3, item_similarity_en= False)
UI_predict_IECombined = prediction(UI_predictionScore_IECombined, 50, rtrain)
# user_item_res1 = evaluate(UI_predict_IECombined, rvalid)
# user_item_res1

100%|████████████████████████████████████████████████████████████████████████████| 6100/6100 [00:02<00:00, 2558.61it/s]


### 4. NO NEED Something random, to be filled

In [55]:
similarity_4 = train(rtrain)
UI_predictionScore_random1 = predict(rtrain_implicit, 100, similarity_4, item_similarity_en= False)
UI_predict_random1 = prediction(UI_predictionScore_random1, 50, rtrain_implicit)
# user_item_res1 = evaluate(UI_predict_IECombined, rvalid)
# user_item_res1

100%|████████████████████████████████████████████████████████████████████████████| 6100/6100 [00:02<00:00, 2714.36it/s]


### 5. something random, to be filled 2

In [56]:
userVisitMatrix = getImplicitMatrix(rtrain)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  This is separate from the ipykernel package so we can avoid doing imports until


In [57]:
similarity1 = train(rtrain)
similarity2 = train(rtrain_implicit)
similarity3 =train(userVisitMatrix)
similarity4 = train(rtrain_userAvg)

In [58]:
UI_predictionScore_max = predictUU(rtrain, 100, 'max', similarity1, similarity2, similarity3, similarity4)
UI_predict_max = prediction(UI_predictionScore_max, 50, rtrain)

100%|████████████████████████████████████████████████████████████████████████████| 6100/6100 [00:02<00:00, 2773.05it/s]


### 6. IU, Explicit

In [59]:
#UU similarity, using cosine similarity
similarity_6 = train(rtrain.T)
#get a user-item matrix  UI prediction
#Predict using UI matrix with ratings in it 
IU_predictionScore_Explicit = predict(rtrain, 100, similarity_6, item_similarity_en= True)
IU_predict_Explicit = prediction(IU_predictionScore_Explicit, 50, rtrain)
#user_item_res1 = evaluate(user_item_predict1, rvalid_userAvg)

100%|████████████████████████████████████████████████████████████████████████████| 6100/6100 [00:02<00:00, 2224.60it/s]


In [60]:
IU_predict_Explicit[0]

array([3655, 2396,  753, 3489,  691, 2235,  292,  883, 3947, 2018, 1450,
       1592, 3222, 1340, 2600,  905,  355, 1520, 1162, 2728, 2663, 3388,
       3237, 1147, 1874, 1547,  953, 3775,  728, 3629, 2080,  835, 1725,
       2715, 3042, 2526, 3983, 2522, 2856, 1798,   36, 3175, 3245, 3564,
       2726, 2887, 1246,  101, 1543, 3546], dtype=int64)

### 7. Item_based TF-IDF

In [61]:
IK_MATRIX = X_cleaned_sparse
I_I_similarity = train(IK_MATRIX)
item_based_prediction_score4 = predict(rtrain, 10, I_I_similarity, item_similarity_en= True)
#for each restuarant top50 users 
Item_predict_tfidf = prediction(item_based_prediction_score4, 50, rtrain)

100%|████████████████████████████████████████████████████████████████████████████| 6100/6100 [00:02<00:00, 2318.81it/s]


In [62]:
Item_predict_tfidf[0]

array([ 863, 3631, 2728,  758,  135,   92, 3994, 3501, 1546, 2128,  223,
       2938, 2666, 1747,  153, 1081,  769, 3663,  828,  227, 2600,   12,
       2755, 3719, 3208, 3411, 3661, 1948, 1922, 2834,  699, 1869, 3164,
        685, 2971,  145, 2153, 3951,  721,  207, 1000, 2877, 2630, 1340,
       1564,  132,  676, 3426, 1960, 1880], dtype=int64)

### 3 Popularity Metrics
#### get the popular items in three ways
1. avg stars
2. number of reviews
3. percentage liked

The small analysis and the map are in the Analyse_3_ways_of_popularities.ipynb

In [63]:
# number of reviews popularity list, redundent with the output of the next method
dff_popular = df.copy()
dff_popular = dff_popular.sort_values(by=["review_count_y"], ascending=False).drop_duplicates(subset = 'business_id', keep = 'first')
#Get the list of restaurants accoridng to their popularity level
popular_list = dff_popular["business_num_id"].tolist()

In [65]:
def get_three_popularity_matrix(df_original,rtrain):
    # get the list of popular items by ranking the number of reviews
    dff_popular = df_original.copy()
    dff_popular = dff_popular.sort_values(by=["review_count_y"], ascending=False).drop_duplicates(subset = 'business_id', keep = 'first')
    popular_list_num_of_reviews = dff_popular["business_num_id"].tolist()
    
    # get the list of popular items by ranking average rating score
    dff_popular_rating = df_original.copy()
    dff_popular_rating = dff_popular.sort_values(by=["business_stars"], ascending=False).drop_duplicates(subset = 'business_id', keep = 'first')
    popular_list_avg_stars = dff_popular_rating["business_num_id"].tolist()
    
    # get the popularity items by using the percentage liked method(number of liked items / total items)
    numUsers = rtrain.shape[0]
    numItems = rtrain.shape[1]
    
    predictionMatrix = np.zeros((numUsers , numItems))

    # Define function for converting 1-5 rating to 0/1 (like / don't like)
    vf = np.vectorize(lambda x: 1 if x >= 4 else 0)
    rtrain_array = rtrain.toarray()
    # For every item calculate the number of people liked (4-5) divided by the number of people that rated
    itemPopularity = np.zeros((numItems))
    for item in range(numItems):
        numOfUsersRated = len(rtrain_array[:, item].nonzero()[0])
        numOfUsersLiked = len(vf(rtrain_array[:, item]).nonzero()[0])
#         if numOfUsersRated == 0:
        # set a threshold to filter out restaurants with very few reviews
        if numOfUsersRated <= 100:
            itemPopularity[item] = 0
        else:
            itemPopularity[item] = numOfUsersLiked/numOfUsersRated
    popular_list_liked_ratio = itemPopularity.argsort()
    
    return np.asarray(popular_list_num_of_reviews),np.asarray(popular_list_avg_stars),popular_list_liked_ratio

In [66]:
# get number of users and number of items
numUsers = rtrain.shape[0]
numItems = rtrain.shape[1]

# get the 1d array of avg. stars, number of reviews and percentage liked ratio for the three popular metric
popular_list_num_of_reviews,popular_list_avg_stars,popular_list_liked_ratio = get_three_popularity_matrix(df,rtrain)

# transfer to a matrix(list * number of users)
matrix_popular_list_num_of_reviews = np.tile(popular_list_num_of_reviews,(numUsers,1))
matrix_popular_list_avg_stars = np.tile(popular_list_avg_stars,(numUsers,1))
matrix_popular_list_liked_ratio = np.tile(popular_list_liked_ratio,(numUsers,1))


### Recommend 3 lists for three locations by using "popular_list_liked_ratio" method

Note that the input of the geographical_dist method can only be list or n-d array

In [68]:
# either a list or an array
def geographical_dist(prediction_matrix,intersection,user_id, bus_indexRange):
    lst = []
    length = 0
    #if the prediction matrix is a list of items for an user 
    if isinstance(prediction_matrix, list):
        length = len(prediction_matrix)
    #loop through the prediction matrix for the user, if passed in a prediction matrix 
    else:
        length = prediction_matrix[user_id].shape[0]
    for j in range(length):
        if isinstance(prediction_matrix, list):
            coordinateDict = yaml.safe_load(df_location[df_location["business_num_id"] == prediction_matrix[j]].iloc[0].coordinates)
        else:    
            coordinateDict = yaml.safe_load(df_location[df_location["business_num_id"] == prediction_matrix[user_id][j]].iloc[0].coordinates)
        test_point = Point(coordinateDict['latitude'],coordinateDict['longitude'])
        
        #Get the distance with the test point
        result = distance.distance(intersection,test_point).kilometers
        if result<=0.6:
            #append the jth item if the condition matches
            if isinstance(prediction_matrix, list):
                lst.append(prediction_matrix[j])
            else:
                lst.append(prediction_matrix[user_id][j])
        lst = lst[0:bus_indexRange]
    return lst

### This step should be at the beginning

In [69]:
# get a copy of df
df_location = df.copy()

# three locations for user input
yonge_and_finch = Point("43.779824, -79.415665")
bloor_and_bathurst = Point("43.665194,-79.411208")
queen_and_spadina = Point("43.648772,-79.396259")

# three locations for user recommendation
bloor_and_yonge = Point("43.670409,-79.386814")
dundas_and_yonge = Point("43.6561,-79.3802")
spadina_and_dundas = Point("43.653004,-79.398082")

# list of percentage liked ratio (transfer the format in order to feed in to the geographical_dist method)
list_popular_liked_ratio = popular_list_liked_ratio.tolist()

In [70]:
# popularity list for input does not need to input the user_id
# put user_id as 0 here but it does not matter
# bus_indexRange = 15, gives a list of 15 restaurants per location
bus_indexRange = 15
yonge_and_finch_list = geographical_dist(list_popular_liked_ratio,yonge_and_finch,0, bus_indexRange)
bloor_and_bathurst_list = geographical_dist(list_popular_liked_ratio,bloor_and_bathurst,0, bus_indexRange)
queen_and_spadina_list = geographical_dist(list_popular_liked_ratio,queen_and_spadina,0, bus_indexRange)


In [71]:
#Construct the dictionary to be used for UI 
#0 is hard coded, not used for now 
dict1_yongeFinch = constructResDictionary(0, bus_indexRange, yonge_and_finch_list)
dict2_bloorBathurst = constructResDictionary(0, bus_indexRange, bloor_and_bathurst_list)
dict3_queenSpadina = constructResDictionary(0, bus_indexRange, queen_and_spadina_list)


#These construct the restaurant name : business id dictionary for initial user setting
RestaurantBusId = {}

# yonge_and_finch_list
# bloor_and_bathurst_list
# queen_and_spadina_list
restYF = list(dict1_yongeFinch.keys())
restBB = list(dict2_bloorBathurst.keys())
restQS = list(dict3_queenSpadina.keys())

#loop through all items
for count in range(len(yonge_and_finch_list)):
    RestaurantBusId[restYF[count]] = yonge_and_finch_list[count]
    RestaurantBusId[restBB[count]] = bloor_and_bathurst_list[count]
    RestaurantBusId[restQS[count]] = queen_and_spadina_list[count]

RestaurantBusId

In [88]:
#Validation 
#Get the business information for the recommended business 
# businessSeries = df[df["business_num_id"] == 1904].iloc[0]
# #Get the business name 
# businessSeries['name']

### Display the UI for user to choose restaurant from 

In [107]:
class MultiListbox_Initialize(Frame):
    
    def __init__(self, master, lists,list1Res,list2Res,list3Res):
        Frame.__init__(self, master)
        self.lists = []
        #self.entries = []
        #self.fields = 'Like', 'Dislike'
        self.dropDownTextBox = 'Rank1','Score1', 'Rank2','Score2','Rank3','Score3'
        #Var for checkbox
        #self.var = IntVar()
        self.var = []
        self.listNames = []
        self.responseDict = {}
        #self.options = ["Jan", "Feb", "Mar"] #etc
        #No 0s or else you won't choose it as liked 
        self.scores = [5, 4, 3, 2, 1]
        #stores the list of restaurants for list 1-3
        self.list1Name = list1Res
        self.list2Name = list2Res
        self.list3Name = list3Res
        #To get response from dropdowns 
        self.rankVar = []
        self.scoreVar = []
        
        #list count 0,1,2
        listCount = 0 
        #Loop through the lists, l is the list label and widthW is the width 
        for l,widthW in lists:
            
            frame = Frame(self); frame.pack(side=LEFT, expand=YES, fill=BOTH)
            
            Label(frame, text=l, borderwidth=1, relief=RAISED).pack(fill=X)
            
            #store the list names 
            self.listNames.append(l)
            
            lb = Listbox(frame, width=widthW, borderwidth=0, selectborderwidth=0,
                 relief=FLAT, exportselection=FALSE)
            lb.pack(expand=YES, fill=BOTH)
            
            self.lists.append(lb)
            
            #lb.bind('<B1-Motion>', lambda e, s=self: s._select(e.y))
            #lb.bind('<Button-1>', lambda e, s=self: s._select(e.y))
            
            #lb.bind('<Leave>', lambda e: 'break')
            
            #lb.bind('<B2-Motion>', lambda e, s=self: s._b2motion(e.x, e.y))
            #lb.bind('<Button-2>', lambda e, s=self: s._button2(e.x, e.y))
             
            for fieldText in self.dropDownTextBox:

                if('Rank' in fieldText):
                    
                    #Get the current list number 
                    if listCount+1 == 1:
                        currentOptions = self.list1Name
                    elif listCount+1 == 2:
                        currentOptions = self.list2Name
                    elif listCount+1 == 3:
                        currentOptions = self.list3Name
                        
                    variable = StringVar(master)
                    variable.set(currentOptions[0]) # default value

                    #row = Frame(self)
                    lab = Label(frame, width=15, text=fieldText, anchor='center')
                    drowDwn = OptionMenu(frame, variable, *currentOptions)
                    
                    #Append the variable set from dropdown 
                    self.rankVar.append(variable)
                    

                    lab.pack(side=TOP,fill=X)
 
                    drowDwn.pack(side=TOP)


                elif('Score' in fieldText ):
                    variable = StringVar(master)
                    variable.set(self.scores[0]) # default value

                    #row = Frame(self)
                    lab = Label(frame, width=15, text=fieldText, anchor='center')
                    drowDwn = OptionMenu(frame, variable, *self.scores)
                    
                    #Append te varaible set from dropdown 
                    self.scoreVar.append(variable)

                    lab.pack(side=TOP, fill=X)
                    drowDwn.pack(side=TOP)
            
            listCount += 1
            
        #Set the submit button  
        b1 = Button(master, text='Submit', command=self.fetch)
        b1.pack(side=BOTTOM, padx=5, pady=5)

        #pakcing the frame
        #frame = Frame(self); frame.pack(side=LEFT, fill=Y)
        
        #packing the label
        #Label(frame, borderwidth=1, relief=RAISED).pack(fill=X)

    def _select(self, y):
        row = self.lists[0].nearest(y)
        self.selection_clear(0, END)
        self.selection_set(row)
        return 'break'

    def _button2(self, x, y):
        for l in self.lists: l.scan_mark(x, y)
        return 'break'

    def _b2motion(self, x, y):
        for l in self.lists: l.scan_dragto(x, y)
        return 'break'

    def _scroll(self, *args):
        for l in self.lists:
            apply(l.yview, args)

    def curselection(self):
        return self.lists[0].curselection()

    def delete(self, first, last=None):
        for l in self.lists:
            l.delete(first, last)

#     def get(self, first, last=None):
#         result = []
#         for l in self.lists:
#             result.append(l.get(first,last))
#         if last: return apply(map, [None] + result)
#         return result

    def index(self, index):
        self.lists[0].index(index)

    def insert(self, index, *elements):
        #Loop through the elements 
        for element in elements:
            i = 0
            for l in self.lists:
                l.insert(index, element[i])
                i = i + 1

    def size(self):
        return self.lists[0].size()

    def see(self, index):
        for l in self.lists:
            l.see(index)

    def selection_anchor(self, index):
        for l in self.lists:
            l.selection_anchor(index)

    def selection_clear(self, first, last=None):
        for l in self.lists:
            l.selection_clear(first, last)

    def selection_includes(self, index):
        return self.lists[0].selection_includes(index)

    def selection_set(self, first, last=None):
        for l in self.lists:
            l.selection_set(first, last)
            
    def fetch(self):
        localDict = {}
        localRankList = list(map((lambda var: var.get()), self.rankVar))
        localScoreList = list(map((lambda var: var.get()), self.scoreVar))
        
        #lop through each selected restaurant and score, construct the dictionary 
        for count in range(len(localRankList)):
            localDict[localRankList[count]] = int(localScoreList[count])
            
        print('Your selected restaurants:', localRankList)
        print('Your scores given to the restaurants', localScoreList)
        print('Your final choice is :', localDict)
         
        self.responseDict = localDict.copy()
   

In [108]:
if __name__ == '__main__':
    #Initiate a frame or master? 
    tk = Tk()
    Label(tk, text='List of restaurants from 3 locations - choose 3 each').pack()
    
    #Creating the multi-listbox object, pass in a tuple of tuple (lists), this will be the list objects
    initialSetuUp = MultiListbox_Initialize(tk, (('Yonge & Finch Intersection', 20), ('Bloor & Bathurst Intersection', 20), \
                            ('Queen & Spadina Intersection', 20)),list(dict1_yongeFinch.keys()),list(dict2_bloorBathurst.keys()),\
                                 list(dict3_queenSpadina.keys()))

    #loop through the length of recommend list, # = 5 
    for index in range(len(dict1_yongeFinch)):
        #First restaurant information to display 
        #Following the restaurants' name 
        restList1 = list(dict1_yongeFinch.keys())[index]
        restList2 = list(dict2_bloorBathurst.keys())[index]
        restList3 = list(dict3_queenSpadina.keys())[index]
        
        initialSetuUp.insert(END, (' ', ' ', ' '))
        
        #Inserting the restaurant names
        initialSetuUp.insert(END, ('%d: %s' % (index + 1, restList1),'%d: %s' % (index + 1, restList2),
                         '%d: %s' % (index + 1, restList3)))
        
        #Looping through each attribute keys - resinfo
        for resinfo in dict1_yongeFinch.get(restList1).keys():
            restList1Info = resinfo + ':' + str(dict1_yongeFinch.get(restList1).get(resinfo,''))
            restList2Info = resinfo + ':' + str(dict2_bloorBathurst.get(restList2).get(resinfo,''))
            restList3Info = resinfo + ':' + str(dict3_queenSpadina.get(restList3).get(resinfo,''))
            
            initialSetuUp.insert(END, (restList1Info, restList2Info, restList3Info))
        
        initialSetuUp.insert(END, ('----------------', '----------------', '----------------'))
    
    initialSetuUp.pack(expand=YES,fill=BOTH)

    tk.mainloop()
    
    #Get user response
    UserInitialResponse = initialSetuUp.responseDict
    
    #store the user response into local file
    #csv_fileName = "UserTestResult{:d}.json".format(userTestNumber)
    #with open('userStudyResults//'+csv_fileName, 'w') as fp:
        #json.dump(response, fp)
    

Your selected restaurants: ['Sang-Ji Fried Bao', 'Pyung Won House', 'Hot Star Large Fried Chicken', 'Mapo Korean BBQ', 'Apiecalypse Now!', 'Jin Dal Lae', 'The Porch Toronto', 'Hong Kong Bistro Cafe', 'b.good']
Your scores given to the restaurants ['5', '5', '5', '5', '5', '5', '5', '5', '5']
Your final choice is : {'Sang-Ji Fried Bao': 5, 'Pyung Won House': 5, 'Hot Star Large Fried Chicken': 5, 'Mapo Korean BBQ': 5, 'Apiecalypse Now!': 5, 'Jin Dal Lae': 5, 'The Porch Toronto': 5, 'Hong Kong Bistro Cafe': 5, 'b.good': 5}


In [109]:
UserInitialResponse
assert len(UserInitialResponse) ==9,"User response has duplicates!"

In [131]:
import statistics 
#Get row lis
rowList = [0] * len(UserInitialResponse)
#Get col list
colList = []
#Get data list
dataList = []

for resName, rating in UserInitialResponse.items():
    #Append the bus_num_id as column values
    #RestaurantBusId mapps restuarnt names to business ids
    colList.append(RestaurantBusId[resName])
    dataList.append(rating)

userAvg = statistics.mean(dataList) 
dataWuserAvgList = np.array(dataList) - np.array([userAvg] * len(UserInitialResponse))+ 0.001
rows = np.array(rowList)
cols = np.array(colList)
data = np.array(dataList)
dataWuser = np.array(dataWuserAvgList) 
print(rows, cols,data)

#Get explicit data
userSetUpMatrix = csr_matrix((data, (rows, cols)), shape=(1, rtrain.shape[1]))
#Get with user rating
userSetUpMatrix_WuserAvg = csr_matrix((dataWuser, (rows, cols)), shape=(1, rtrain.shape[1]))

#Generate Implicit user rating vector
implicitUserSetUpMtx = csr_matrix((data, (rows, cols)), shape=(1, rtrain.shape[1]))
implicitUserSetUpMtx[(userSetUpMatrix > 3).nonzero()] = 1

[0 0 0 0 0 0 0 0 0] [2661 2697 2939 2632 2951 2950 2612 2656 2654] [5 5 5 5 5 5 5 5 5]


In [136]:
def csr_vappend(a,b):
    """ Takes in 2 csr_matrices and appends the second one to the bottom of the first one. 
    Much faster than scipy.sparse.vstack but assumes the type to be csr and overwrites
    the first matrix instead of copying it. The data, indices, and indptr still get copied."""

    a.data = np.hstack((a.data,b.data))
    a.indices = np.hstack((a.indices,b.indices))
    a.indptr = np.hstack((a.indptr,(b.indptr + a.nnz)[1:]))
    a._shape = (a.shape[0]+b.shape[0],b.shape[1])
    return a

## Only run the below cell once!

In [155]:
#This would overwrite rtrain,rtrain_implicit, and rtrain_WuserAvg
csr_vappend(rtrain,userSetUpMatrix)
csr_vappend(rtrain_implicit, implicitUserSetUpMtx)
csr_vappend(rtrain_userAvg, userSetUpMatrix_WuserAvg)

<6101x3998 sparse matrix of type '<class 'numpy.float64'>'
	with 179729 stored elements in Compressed Sparse Row format>

#before combine
rtrain
<6100x3998 sparse matrix of type '<class 'numpy.float32'>'
	with 179720 stored elements in Compressed Sparse Row format>
after combine:
rtrain
<6101x3998 sparse matrix of type '<class 'numpy.float64'>'
	with 179729 stored elements in Compressed Sparse Row format>
    
(6099, 19)	5.0
(6099, 22)	3.0
(6099, 323)	3.0
(6099, 778)	4.0
(6099, 1144)	4.0
(6099, 1161)	4.0
(6099, 1525)	4.0
(6099, 1782)	5.0
(6099, 2029)	4.0
(6099, 2572)	4.0
(6099, 2702)	4.0
(6099, 2870)	4.0
(6099, 2940)	3.0
(6099, 2987)	3.0
(6099, 3388)	3.0
(6099, 3816)	5.0

(6100, 2612)	5.0
  (6100, 2632)	5.0
  (6100, 2654)	5.0
  (6100, 2656)	5.0
  (6100, 2661)	5.0
  (6100, 2697)	5.0
  (6100, 2939)	5.0
  (6100, 2950)	5.0
  (6100, 2951)	5.0
  
Before 
rtrain_implicit
<6100x3998 sparse matrix of type '<class 'numpy.float32'>'
	with 116024 stored elements in Compressed Sparse Row format>
<6101x3998 sparse matrix of type '<class 'numpy.float64'>'
	with 116033 stored elements in Compressed Sparse Row format>
    
rtrain_userAvg
<6100x3998 sparse matrix of type '<class 'numpy.float32'>'
	with 179720 stored elements in Compressed Sparse Row format>
<6101x3998 sparse matrix of type '<class 'numpy.float64'>'
	with 179729 stored elements in Compressed Sparse Row format>

## I am going to move this part to before computing similarities 

In [163]:
#This should be the user index
rtrain.shape[0]

6101

#### list 1-3 demo for user input

In [306]:
df_location_yonge_and_finch=df_location[df_location["business_num_id"].isin(yonge_and_finch_list)].drop_duplicates(subset = 'business_num_id', keep = 'first')[["business_num_id","name", "price", "categories","business_stars","review_count_y"]].rename(columns={'review_count_y': 'review_count'})
df_location_yonge_and_finch.head(2)

,business_num_id,name,price,categories,business_stars,review_count
1851,2069,Pastel Creperies & Dessert House,$$,"[{'alias': 'creperies', 'title': 'Creperies'}, {'alias': 'desserts', 'title': 'Desserts'}, {'alias': 'coffee', 'title': 'Coffee & Tea'}]",4.0,316
18077,1943,Sweet O'clock,$,"[{'alias': 'desserts', 'title': 'Desserts'}, {'alias': 'tea', 'title': 'Tea Rooms'}, {'alias': 'icecream', 'title': 'Ice Cream & Frozen Yogurt'}]",3.0,86


In [305]:
pd.set_option('display.max_columns', None)
df_location_bloor_and_bathurst_list=df_location[df_location["business_num_id"].isin(bloor_and_bathurst_list)].drop_duplicates(subset = 'business_num_id', keep = 'first')[["business_num_id","name","price","categories","business_stars","review_count_y"]].rename(columns={'review_count_y': 'review_count'})
df_location_bloor_and_bathurst_list.head(2)

,business_num_id,name,price,categories,business_stars,review_count
12180,2125,The Burger's Priest,$$,"[{'alias': 'burgers', 'title': 'Burgers'}]",3.0,31
29885,1926,Smoke's Poutinerie,$,"[{'alias': 'vegetarian', 'title': 'Vegetarian'}, {'alias': 'tradamerican', 'title': 'American (Traditional)'}]",3.0,65


In [304]:
df_location_queen_and_spadina_list=df_location[df_location["business_num_id"].isin(queen_and_spadina_list)].drop_duplicates(subset = 'business_num_id', keep = 'first')[["business_num_id","name","price","categories","business_stars","review_count_y"]].rename(columns={'review_count_y': 'review_count'})
df_location_queen_and_spadina_list.head(2)

,business_num_id,name,price,categories,business_stars,review_count
2147,2468,Mana'ish Global Flatbread Cafe,$$,"[{'alias': 'lebanese', 'title': 'Lebanese'}, {'alias': 'cafes', 'title': 'Cafes'}, {'alias': 'mediterranean', 'title': 'Mediterranean'}]",4.5,33
8799,2467,The Keg Steakhouse & Bar,$$$,"[{'alias': 'steak', 'title': 'Steakhouses'}, {'alias': 'bars', 'title': 'Bars'}]",4.0,95


### Recommendation Section: Produce the list of restaurants close to the set location

### Initial Setting, don't need to modify

In [150]:
# dff = df.copy()
# dff_popular = df.copy()
# dff_popular = dff_popular.sort_values(by=["review_count_y"], ascending=False).drop_duplicates(subset = 'business_id', keep = 'first')
# popular_list = dff_popular["business_num_id"].tolist()
# dundas_and_yonge = Point("43.6561,-79.3802")
# bay_and_queens = Point("43.6518,-79.3802")
# king_and_jarvis = Point("43.650577,-79.371887")
# bloor_and_yonge = Point("43.670409,-79.386814")
# yonge_and_eglinton = Point("43.7064,-79.3986")

#### Remember to manually change this userIndex!!!

In [278]:
# three locations for user recommendation
intersection = [bloor_and_yonge,dundas_and_yonge,spadina_and_dundas]
# get 3 items for each location
businessIndexRange = 3
# 5 metrics and store them in a list
metric = [UI_predict_Implicit, popular_list, UI_predict_IECombined, IU_predict_Explicit, Item_predict_tfidf]

# manually enter this number!!!!!!
userIndex = 0

#Need this number to perform the user test, so don't repeat, don't mess up
userTestNumber = 0

#### Get the recommendations for each metric using loops, store in the res

In [154]:
# get 3 top recommendations for each locations
# store three locations in one list for each metric

#loop through the 5 recommendation algorithms, initialize the res list 
res = [[] for i in range(len(metric))]
#Loop through the intersections 
for i in range(len(intersection)):
    #loop through each metric 
    for j in range(len(metric)):
        temp = geographical_dist(metric[j],intersection[i],userIndex,businessIndexRange)
        res[j] += temp

In [155]:
# top 10 recommendations for each metric
# the sequence is -> [UI_predict_Implicit, popular_list, UI_predict_IECombined, IU_predict_Explicit, Item_predict_tfidf]
# get three recommendations for each location, so the len for each row is 3*3
res

[[1248, 883, 2743, 2156, 2856, 1912, 3672],
 [1253, 1248, 1552, 1795, 1798, 36, 2934, 1488, 1547],
 [1248, 883, 2743, 2156, 2856, 2728, 1912],
 [883, 1162, 2715, 1592, 2728, 3388],
 [135, 12, 1880, 699, 2728, 2666, 1869]]

#### Look back the the origal prediction matrix and rearrange the recommendations list, choose the top 3

In [161]:
res_final = []
#loop through the elements in the res list (5 recommend lists)
for element in range(len(res)):
    dic = {}
    #loop through each element in the recomemndation list (order:element)
    for i in range(len(res[element])):
        #if the recommendation is a list 
        if isinstance(metric[element], list):
            dic[metric[element].index(res[element][i])] = res[element][i]
        #if the recommendation is a matrix 
        else:
            dic[metric[element][userIndex].tolist().index(res[element][i])] = res[element][i]
    temp = []
    for j in sorted(dic.keys()):
        temp.append(int(dic[j]))
    res_final.append(temp[0:3])

In [160]:
# the final recommendations for each metric
res_final

[[2856, 883, 1912],
 [1795, 2934, 1488],
 [883, 2856, 1248],
 [883, 1592, 1162],
 [2728, 135, 2666]]

## Reminder!!!! manually adjust userIndex after inserting a new row of user preference!!!

### Produce the list of restaurants close to the set location

In [41]:
# #First list
# UI_predict_Explicit = geographical_dist(UI_predict_Explicit,intersection,userIndex,businessIndexRange)

# #Second list
# UI_predict_Implicit = geographical_dist(UI_predict_Implicit,intersection,userIndex,businessIndexRange)

# #Third list
# UI_predict_IECombined = geographical_dist(UI_predict_IECombined,intersection,userIndex,businessIndexRange)

# #Fourth lsit
# UI_predict_popular = geographical_dist(popular_list,intersection,userIndex,businessIndexRange)

# #Five list
# UI_predict_max = geographical_dist(UI_predict_max,intersection,userIndex,businessIndexRange)

In [42]:
# UI_predict_Explicit

[2378, 2584, 103, 3482, 1893]

In [43]:
# UI_predict_Implicit

[2378, 3035, 1441, 3626, 746]

In [44]:
# UI_predict_popular

[3482, 2378, 1780, 2921, 1480]

In [45]:
# UI_predict_max

[2378, 3482, 3649, 3725, 1441]

In [46]:
#df[df["business_num_id"] == UI_prediction[userIndex][busIndex]].iloc[0]

## Match restaurant information according to business_num_id

#### Current sequence: UI_predict_Implicit, popular_list, UI_predict_IECombined, IU_predict_Explicit, Item_predict_tfidf

In [162]:
#Enter the user number that we are trying to recommend for
#Enter the # of businesses that we want to recommend for them 
#Pass in the UI_Prediction matrix for users 
#userIndex parameter not used 
def constructResDictionary(userIndex, busIndexRange, UI_prediction):
    #Construct the dictionary for the recommended restaurants to display 
    dictionaryToConstruct = {}
    
    #Loop through the number of businesses 
    for busIndex in range(busIndexRange):
        #Get the business information for the recommended business 
        businessSeries = df[df["business_num_id"] == UI_prediction[busIndex]].iloc[0]
        #Get the business name 
        busName = businessSeries['name']
        
        #get the list of strings to generate the address information 
        address_generator = (str(w) for w in yaml.safe_load(businessSeries.location)['display_address'])
        busLocation = ', '.join(address_generator)
        bus_Price = businessSeries.price
        busStars = businessSeries.business_stars
        busReviewCount = businessSeries.review_count_y 
        category_generator = (str(s) for s in [cateDict['title'] for cateDict in yaml.safe_load(businessSeries.categories)])
        #busCategories = [cateDict['title'] for cateDict in yaml.safe_load(businessSeries.categories)]
        busCategories = ', '.join(category_generator)
        #Now add the restaurant to the dictionary
        dictionaryToConstruct[busName] = {'Address': busLocation,\
                                'Price': bus_Price,\
                                 'Star': busStars, \
                                 'Review Count': busReviewCount, \
                                 'Category': busCategories}
    return dictionaryToConstruct

In [163]:
#Trying to recommend for user 0 for now 


dict1_ExplicitRecommend = {}
dict2_ImplicitRecommend = {}
dict3_EICombineRecommend = {}
dict4_PopularityRecommend = {}
dict5_maxRecommend = {}
#User predict [user index] [item index]
#Add a for loop for the top recommended items

dict1_ExplicitRecommend = constructResDictionary(userIndex, businessIndexRange, res_final[0])
dict2_ImplicitRecommend = constructResDictionary(userIndex, businessIndexRange, res_final[1])
dict3_EICombineRecommend = constructResDictionary(userIndex, businessIndexRange, res_final[2])
dict4_PopularityRecommend = constructResDictionary(userIndex, businessIndexRange, res_final[3])
dict5_maxRecommend = constructResDictionary(userIndex, businessIndexRange, res_final[4])

#Say we are recommending 5 restaurants for now
# for busIndex in range(5):
#     businessSeries = df[df["business_num_id"] == UI_predict_Explicit[0][busIndex]].iloc[0]
#     busName = businessSeries['name']
#     address_generator = (str(w) for w in yaml.safe_load(businessSeries.location)['display_address'])
#     busLocation = ', '.join(address_generator)
#     bus_Price = businessSeries.price
#     busStars = businessSeries.business_stars
#     busReviewCount = businessSeries.review_count_y 
#     category_generator = (str(s) for s in [cateDict['title'] for cateDict in yaml.safe_load(businessSeries.categories)])
#     #busCategories = [cateDict['title'] for cateDict in yaml.safe_load(businessSeries.categories)]
#     busCategories = ', '.join(category_generator)
#     #Now add the restaurant to the dictionary
#     dict1stAlgo[busName] = {'Address': busLocation,\
#                             'Price': bus_Price,\
#                              'Star': busStars, \
#                              'Review Count': busReviewCount, \
#                              'Category': busCategories}

### Multilist box with entries

In [283]:
class MultiListbox_entries(Frame):
    
    def __init__(self, master, lists):
        Frame.__init__(self, master)
        self.lists = []
        self.entries = []
        self.fields = 'Like', 'Dislike'
        #Var for checkbox
        #self.var = IntVar()
        self.var = []
        self.listNames = []
        self.responseDict = {}
        
        #Loop through the lists, l is the list label and widthW is the width 
        for l,widthW in lists:
            
            frame = Frame(self); frame.pack(side=LEFT, expand=YES, fill=BOTH)
            
            Label(frame, text=l, borderwidth=1, relief=RAISED).pack(fill=X)
            
            #store the list names 
            self.listNames.append(l)
            
            lb = Listbox(frame, width=widthW, borderwidth=0, selectborderwidth=0,
                 relief=FLAT, exportselection=FALSE)
            lb.pack(expand=YES, fill=BOTH)
            
            self.lists.append(lb)
            
            lb.bind('<B1-Motion>', lambda e, s=self: s._select(e.y))
            lb.bind('<Button-1>', lambda e, s=self: s._select(e.y))
            
            lb.bind('<Leave>', lambda e: 'break')
            
            lb.bind('<B2-Motion>', lambda e, s=self: s._b2motion(e.x, e.y))
            lb.bind('<Button-2>', lambda e, s=self: s._button2(e.x, e.y))
             
             #loop through the fields to 
            for field in fields:
                #row = Frame(self)
                lab = Label(frame, width=15, text=field, anchor='w')
                ent = Entry(frame)
                #row.pack(side=BOTTOM, fill=Y, padx=5, pady=5) #=BOTTOM
                lab.pack()
                #ent.pack(side=RIGHT, expand=YES, fill=Y)
                ent.pack(side=TOP, fill=X)
                self.entries.append((field, ent))
            
            localVar = IntVar()
            
            self.var.append(localVar)
            
            #Checkbox 
            #c = Checkbutton(frame, text="Liked", variable=localVar, command=self.cb(count))
            c = Checkbutton(frame, text="Liked", variable=localVar)
            c.pack()
            
        print(self.var)
        #Set the submit button 
        #master.bind('<Button-1>', (lambda event, e=self.entries: fetch(e)))   
        #b1 = Button(master, text='Submit', command=(lambda e=self.entries: self.fetch(e)))
        b1 = Button(master, text='Submit', command=self.fetchDict)
        b1.pack(side=BOTTOM, padx=5, pady=5)
        #b2 = Button(root, text='Quit', command=root.quit)
        #b2.pack(side=LEFT, padx=5, pady=5)
            
        #pakcing the frame
        frame = Frame(self); frame.pack(side=LEFT, fill=Y)
        
        #packing the label
        Label(frame, borderwidth=1, relief=RAISED).pack(fill=X)
        #Setting a scrollbar 
#         sb = Scrollbar(frame, orient=VERTICAL, command=self._scroll)
#         sb.pack(expand=YES, fill=Y)
#         self.lists[0]['yscrollcommand']=sb.set

    def _select(self, y):
        row = self.lists[0].nearest(y)
        self.selection_clear(0, END)
        self.selection_set(row)
        return 'break'
    
    #For checkbox 
    def cb(self, index):
        print ("variable is", self.var[index].get())

    def _button2(self, x, y):
        for l in self.lists: l.scan_mark(x, y)
        return 'break'

    def _b2motion(self, x, y):
        for l in self.lists: l.scan_dragto(x, y)
        return 'break'

    def _scroll(self, *args):
        for l in self.lists:
            apply(l.yview, args)

    def curselection(self):
        return self.lists[0].curselection()

    def delete(self, first, last=None):
        for l in self.lists:
            l.delete(first, last)

#     def get(self, first, last=None):
#         result = []
#         for l in self.lists:
#             result.append(l.get(first,last))
#         if last: return apply(map, [None] + result)
#         return result

    def index(self, index):
        self.lists[0].index(index)

    def insert(self, index, *elements):
        #Loop through the elements 
        for element in elements:
            i = 0
            for l in self.lists:
                l.insert(index, element[i])
                i = i + 1

    def size(self):
        return self.lists[0].size()

    def see(self, index):
        for l in self.lists:
            l.see(index)

    def selection_anchor(self, index):
        for l in self.lists:
            l.selection_anchor(index)

    def selection_clear(self, first, last=None):
        for l in self.lists:
            l.selection_clear(first, last)

    def selection_includes(self, index):
        return self.lists[0].selection_includes(index)

    def selection_set(self, first, last=None):
        for l in self.lists:
            l.selection_set(first, last)
            
    def fetch(self,entries):
        count = 1
        
        for entry in entries:
            if count <= 2:
                listCount = 1
            elif count <= 4:
                listCount = 2
            elif count <= 6:
                listCount = 3
            elif count <= 8:
                listCount = 4
            elif count <= 10:
                listCount = 5             
            #print on odd number counts
            if count %2 != 0:
                print('list:', listCount)
            #index 0 is the field 
            #index 1 is the entry data 
            field = entry[0]
            text  = entry[1].get()
            print('%s: "%s"' % (field, text))
            
            count =count + 1
            
        print(list(map((lambda var: var.get()), self.var)))
   
    #This function returns a dictionary of the results
    def fetchDict(self):
        responseDict = {}
        count = 1
        
        likeResult = list(map((lambda var: var.get()), self.var))
        
        for entry in self.entries:
            tempDict = {}
            
            field = entry[0]
            text  = entry[1].get()
            # like:"text content"
            tempDict[field] = text
            
            if count <= 2:
                listCount = 1
            elif count <= 4:
                listCount = 2
            elif count <= 6:
                listCount = 3
            elif count <= 8:
                listCount = 4
            elif count <= 10:
                listCount = 5      
                
            #update the dictionary with corresponding listname :  
            try:
                responseDict[self.listNames[listCount-1]].update(tempDict) 
            except:
                responseDict[self.listNames[listCount-1]] = tempDict 
            
            if count %2 != 0:
                #print on odd number counts
                print('list:', listCount)
                #Get the hit indicator and update the dictionary 
                tempLikeDict = {'hit': likeResult[listCount-1]}
                responseDict[self.listNames[listCount-1]].update(tempLikeDict)
            
            #Print the current the field and entry text
            print('%s: "%s"' % (field, text))
           
            count =count + 1
        
        #print the hit list
        print(list(map((lambda var: var.get()), self.var)))

        self.responseDict = responseDict.copy()
        
        
    #Make the entry form 
    def makeform(root, fields):
        #For each field, like 
        for field in fields:
            row = Frame(master)
            lab = Label(row, width=15, text=field, anchor='w')
            ent = Entry(row)
            row.pack(side=TOP, fill=X, padx=5, pady=5)
            lab.pack(side=LEFT)
            ent.pack(side=RIGHT, expand=YES, fill=X)
            self.entries.append((field, ent))
            
        self.entries =  entries

### Sequence for now: UI_predict_Implicit, popular_list, UI_predict_IECombined, IU_predict_Explicit, Item_predict_tfidf 

In [405]:
if __name__ == '__main__':
    #Initiate a frame or master? 
    tk = Tk()
    Label(tk, text='List of recommended restaurants').pack()
    
    #Creating the multi-listbox object, pass in a tuple of tuple (lists), this will be the list objects
    mlb = MultiListbox_entries(tk, (('Implicit User-Rating', 20), ('Popularity List', 20), \
                            ('Explicit Implicit Combined', 20), ('Explicit Item-Rating', 20), ('TF-IDF Score of Item', 20)))

    #loop through the length of recommend list, # = 5 
    for index in range(len(dict1_ExplicitRecommend)):
        #First restaurant information to display 
        #Following the restaurants' name 
        restList1 = list(dict1_ExplicitRecommend.keys())[index]
        restList2 = list(dict2_ImplicitRecommend.keys())[index]
        restList3 = list(dict3_EICombineRecommend.keys())[index]
        restList4 = list(dict4_PopularityRecommend.keys())[index]
        restList5 = list(dict5_maxRecommend.keys())[index]
        
        mlb.insert(END, (' ', ' ', ' ', ' ', ' '))
        
        #Inserting the restaurant names
        mlb.insert(END, ('%d: %s' % (index + 1, restList1),'%d: %s' % (index + 1, restList2),
                         '%d: %s' % (index + 1, restList3),'%d: %s' % (index + 1, restList4),
                         '%d: %s' % (index + 1, restList5)))
        
        #Looping through each attribute keys - resinfo
        for resinfo in dict1_ExplicitRecommend.get(restList1).keys():
            restList1Info = resinfo + ':' + str(dict1_ExplicitRecommend.get(restList1).get(resinfo,''))
            restList2Info = resinfo + ':' + str(dict2_ImplicitRecommend.get(restList2).get(resinfo,''))
            restList3Info = resinfo + ':' + str(dict3_EICombineRecommend.get(restList3).get(resinfo,''))
            restList4Info = resinfo + ':' + str(dict4_PopularityRecommend.get(restList4).get(resinfo,''))
            restList5Info = resinfo + ':' + str(dict5_maxRecommend.get(restList5).get(resinfo,''))
            
            mlb.insert(END, (restList1Info, restList2Info, restList3Info, restList4Info, restList5Info))
        
        mlb.insert(END, ('----------------', '----------------', '----------------', '----------------', '----------------'))
    
    mlb.pack(expand=YES,fill=BOTH)

    tk.mainloop()
    
    #Get user response
    response = mlb.responseDict
    
    #store the user response into local file
    csv_fileName = "UserTestResult{:d}.json".format(userTestNumber)
    with open('userStudyResults//'+csv_fileName, 'w') as fp:
        json.dump(response, fp)
    

[<tkinter.IntVar object at 0x00000147670A2438>, <tkinter.IntVar object at 0x0000014767A67860>, <tkinter.IntVar object at 0x000001477FCC2D68>, <tkinter.IntVar object at 0x00000147693CB8D0>, <tkinter.IntVar object at 0x0000014768B26B38>]


### I get the user response information here, so please make sure the sequence for list names are correct when initializing our mlb object!

In [280]:
# csv_fileName = "UserTestResult{:d}.json".format(userTestNumber)
# with open(csv_fileName, 'w') as fp:
#     json.dump(response, fp)

In [281]:
#To load the result file with json 
# with open(csv_fileName, 'r') as fp:
#     test = json.load(fp)

In [202]:
#Get the 0th user review data 
rows, cols = rtrain[0].nonzero()
cols

array([  38,  155,  290,  310,  333,  384,  435,  440,  768,  882,  916,
       1089, 1102, 1212, 1218, 1488, 1529, 1710, 1772, 1953, 2012, 2192,
       2343, 2518, 2622, 2760, 2873, 2934, 2945, 3031, 3059, 3083, 3199,
       3309, 3376, 3412, 3524, 3540, 3791, 3866])

In [210]:
#Get the corresponding business information using the business num id 
pd.set_option('display.max_colwidth', -1)
df[df["business_num_id"].isin(cols)].drop_duplicates(subset = 'business_num_id', keep = 'first')[["business_num_id","name","categories","business_stars","review_count_y"]].rename(columns={'review_count_y': 'review_count'})

,business_num_id,name,categories,business_stars,review_count
8263,2945,Mayrik,"[{'alias': 'mediterranean', 'title': 'Mediterranean'}, {'alias': 'mideastern', 'title': 'Middle Eastern'}]",4.5,31
9368,1488,Banh Mi Boys,"[{'alias': 'sandwiches', 'title': 'Sandwiches'}]",4.5,1049
11498,2012,Pray Tell,"[{'alias': 'cocktailbars', 'title': 'Cocktail Bars'}, {'alias': 'tapasmallplates', 'title': 'Tapas/Small Plates'}, {'alias': 'breakfast_brunch', 'title': 'Breakfast & Brunch'}]",4.0,54
11674,333,El Rey Mezcal Bar,"[{'alias': 'cocktailbars', 'title': 'Cocktail Bars'}, {'alias': 'gastropubs', 'title': 'Gastropubs'}, {'alias': 'mexican', 'title': 'Mexican'}]",3.5,49
12569,2873,Odin,"[{'alias': 'cafes', 'title': 'Cafes'}, {'alias': 'bars', 'title': 'Bars'}]",3.5,63
15311,1529,Si Lom Thai Bistro,"[{'alias': 'thai', 'title': 'Thai'}]",4.0,108
16684,768,Mother's Dumplings,"[{'alias': 'chinese', 'title': 'Chinese'}]",3.0,478
18288,2934,Seven Lives Tacos Y Mariscos,"[{'alias': 'mexican', 'title': 'Mexican'}]",4.5,1159
23246,3412,Bar Isabel,"[{'alias': 'spanish', 'title': 'Spanish'}, {'alias': 'tapas', 'title': 'Tapas Bars'}]",4.0,379
32766,3524,Constantine,"[{'alias': 'mediterranean', 'title': 'Mediterranean'}, {'alias': 'mideastern', 'title': 'Middle Eastern'}, {'alias': 'italian', 'title': 'Italian'}]",3.5,61


## Tkinter examples

In [ ]:
#!/usr/bin/python3
#Multiple checkboxes example

# from tkinter import *
# class Checkbar(Frame):
#    def __init__(self, parent=None, picks=[], side=LEFT, anchor=W):
#       Frame.__init__(self, parent)
#       self.vars = []
    
#       for pick in picks:
#          var = IntVar()
#          chk = Checkbutton(self, text=pick, variable=var)
#          chk.pack(side=side, anchor=anchor, expand=YES)
#          self.vars.append(var)
            
#    def state(self):
#       return map((lambda var: var.get()), self.vars)
    
# if __name__ == '__main__':
#    root = Tk()
#    lng = Checkbar(root, ['Python', 'Ruby', 'Perl', 'C++'])
#    tgl = Checkbar(root, ['English','German'])
#    lng.pack(side=TOP,  fill=X)
#    tgl.pack(side=LEFT)
#    lng.config(relief=GROOVE, bd=2)

#    def allstates(): 
#       print(list(lng.state()), list(tgl.state()))
#    Button(root, text='Quit', command=root.quit).pack(side=RIGHT)
#    Button(root, text='Peek', command=allstates).pack(side=RIGHT)
#    root.mainloop()

In [ ]:
# #For checkbox demo 
# class checkBox(Frame):
    
#     def __init__(self, master):
#             self.var = IntVar()
            
#             c = Checkbutton(master, text="Liked", variable=self.var, command=self.cb)
#             c.pack()

#     def cb(self):
#         print ("variable is", self.var.get())

# if __name__ == '__main__':
#     #Initialize the root 
#     root = Tk()

#     #Initialize the entries
#     checkbox = checkBox(root)

#     root.mainloop()

In [ ]:
# #Demo for text boxes
# fields = ['Like', 'Dislike']

# def fetch(entries):
#     for entry in entries:
#         #index 0 is the field 
#         #index 1 is the entry data 
#         field = entry[0]
#         text  = entry[1].get()
#         print('%s: "%s"' % (field, text)) 

# def makeform(root, fields):
#     entries = []
#     #For each field, like 
#     for field in fields:
#         row = Frame(root)
#         lab = Label(row, width=15, text=field, anchor='w')
#         ent = Entry(row)
#         row.pack(side=TOP, fill=X, padx=5, pady=5)
#         lab.pack(side=LEFT)
#         ent.pack(side=RIGHT, expand=YES, fill=X)
#         entries.append((field, ent))
#     return entries

# if __name__ == '__main__':
#     #Initialize the root 
#     root = Tk()
    
#     #Initialize the entries
#     ents = makeform(root, fields)
    
#     #the buttons 
#     root.bind('<Button-1>', (lambda event, e=ents: fetch(e)))   
#     b1 = Button(root, text='Show', command=(lambda e=ents: fetch(e)))
#     b1.pack(side=LEFT, padx=5, pady=5)
#     b2 = Button(root, text='Quit', command=root.quit)
#     b2.pack(side=LEFT, padx=5, pady=5)
#     root.mainloop()

In [285]:
#Initial Version
# class MultiListbox(Frame):
    
#     def __init__(self, master, lists):
#         Frame.__init__(self, master)
#         self.lists = []
        
#         #Loop through the lists, l is the list label and widthW is the width 
#         for l,widthW in lists:
            
#             frame = Frame(self); frame.pack(side=LEFT, expand=YES, fill=BOTH)
            
#             Label(frame, text=l, borderwidth=1, relief=RAISED).pack(fill=X)
            
#             lb = Listbox(frame, width=widthW, borderwidth=0, selectborderwidth=0,
#                  relief=FLAT, exportselection=FALSE)
#             lb.pack(expand=YES, fill=BOTH)
            
#             self.lists.append(lb)
            
#             lb.bind('<B1-Motion>', lambda e, s=self: s._select(e.y))
#             lb.bind('<Button-1>', lambda e, s=self: s._select(e.y))
            
#             lb.bind('<Leave>', lambda e: 'break')
            
#             lb.bind('<B2-Motion>', lambda e, s=self: s._b2motion(e.x, e.y))
#             lb.bind('<Button-2>', lambda e, s=self: s._button2(e.x, e.y))
            
#         #pakcing the frame
#         frame = Frame(self); frame.pack(side=LEFT, fill=Y)
        
#         #packing the label
#         Label(frame, borderwidth=1, relief=RAISED).pack(fill=X)
        
#         #Setting a scrollbar 
# #         sb = Scrollbar(frame, orient=VERTICAL, command=self._scroll)
        
# #         sb.pack(expand=YES, fill=Y)
        
# #         self.lists[0]['yscrollcommand']=sb.set

#     def _select(self, y):
#         row = self.lists[0].nearest(y)
#         self.selection_clear(0, END)
#         self.selection_set(row)
#         return 'break'

#     def _button2(self, x, y):
#         for l in self.lists: l.scan_mark(x, y)
#         return 'break'

#     def _b2motion(self, x, y):
#         for l in self.lists: l.scan_dragto(x, y)
#         return 'break'

#     def _scroll(self, *args):
#         for l in self.lists:
#             apply(l.yview, args)

#     def curselection(self):
#         return self.lists[0].curselection()

#     def delete(self, first, last=None):
#         for l in self.lists:
#             l.delete(first, last)

# #     def get(self, first, last=None):
# #         result = []
# #         for l in self.lists:
# #             result.append(l.get(first,last))
# #         if last: return apply(map, [None] + result)
# #         return result

#     def index(self, index):
#         self.lists[0].index(index)

#     def insert(self, index, *elements):
#         #Loop through the elements 
#         for element in elements:
#             i = 0
#             for l in self.lists:
#                 l.insert(index, element[i])
#                 i = i + 1

#     def size(self):
#         return self.lists[0].size()

#     def see(self, index):
#         for l in self.lists:
#             l.see(index)

#     def selection_anchor(self, index):
#         for l in self.lists:
#             l.selection_anchor(index)

#     def selection_clear(self, first, last=None):
#         for l in self.lists:
#             l.selection_clear(first, last)

#     def selection_includes(self, index):
#         return self.lists[0].selection_includes(index)

#     def selection_set(self, first, last=None):
#         for l in self.lists:
#             l.selection_set(first, last)

In [ ]:
# #Dropdown mdemo
# OPTIONS = [
# "Jan",
# "Feb",
# "Mar"
# ] #etc

# master = Tk()

# variable = StringVar(master)
# variable.set(OPTIONS[0]) # default value

# w = OptionMenu(master, variable, *OPTIONS)
# w.pack()

# def ok():
#     print ("value is:" + variable.get())

# #retrive the drop down value 
# button = Button(master, text="OK", command=ok)
# button.pack()

# mainloop()